In [1]:
import pandas as pd
import requests
import numpy as np
from bs4 import BeautifulSoup
from time import sleep
from random import randint

In [2]:
array_of_df = []

# Collecting data of each year from 1990 to 2021

for Year in range(1990, 2022): 
    sleep (randint(2,4))
    
    # Get the url of the specific year
    
    URL = f'https://www.basketball-reference.com/leagues/NBA_{Year}_totals.html'
    response = requests.get(URL)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Get table from page
    
    tbl = soup.find_all("table",id = "totals_stats")[0]
    
    # Get headers from table
    
    headers = [th.getText() for th in tbl.thead.findAll('th')]
    
    # Remove Rank column - not needed
    
    headers = headers[1:]
    
    # Get rows without first header row
    
    rows = soup.findAll('tr')[1:]
    
    # Get all cells
    
    player_stats = [[td.getText() for td in rows[i].findAll('td')] for i in range(len(rows))]
    
    # Create dataframe from data
    
    df = pd.DataFrame(player_stats, columns = headers)
    
    # Insert the Year columm to the DF
    
    df.insert(0,'Year',Year)
    
    # Drop duplicated players in the same season, keep the ones with the most games played
    
    df.drop_duplicates(subset=['Player'], keep = 'first', inplace = True)
    
    # Insert this years dataframe to the array of dataframes
    
    array_of_df.insert(0,df)

### Combining all data frames of each year to 1 big dataframe.

In [3]:
stats_df = pd.concat(array_of_df)
stats_df

Year             Player Pos Age   Tm   G  GS    MP   FG   FGA  ...  \
0    2021   Precious Achiuwa  PF  21  MIA  61   4   737  124   228  ...   
1    2021       Jaylen Adams  PG  24  MIL   7   0    18    1     8  ...   
2    2021       Steven Adams   C  27  NOP  58  58  1605  189   308  ...   
3    2021        Bam Adebayo   C  23  MIA  64  64  2143  456   800  ...   
4    2021  LaMarcus Aldridge   C  35  TOT  26  23   674  140   296  ...   
..    ...                ...  ..  ..  ...  ..  ..   ...  ...   ...  ...   
472  1990  Orlando Woolridge  SF  30  LAL  62   2  1421  306   550  ...   
473  1990   Haywoode Workman  PG  24  ATL   6   0    16    2     3  ...   
474  1990      James Worthy*  SF  28  LAL  80  80  2960  711  1298  ...   
475  1990        Danny Young  PG  27  POR  82   8  1393  138   328  ...   
477  1990      Michael Young  SF  29  LAC  45   2   459   92   194  ...   

       FT%  ORB  DRB  TRB  AST STL BLK  TOV   PF   PTS  
0     .509   73  135  208   29  20  28   43   91   304  
1             0    3    3    2   0   0    0    1     2  
2     .444  213  301  514  111  54  38   78  113   438  
3     .799  142  431  573  346  75  66  169  145  1197  
4     .872   19   99  118   49  11  29   27   47   352  
..     ...  ...  ...  ...  ...  ..  ..  ...  ...   ...  
472   .733   49  136  185   96  39  46   73  160   788  
473  1.000    0    3    3    2   3   0    0    3     6  
474   .782  160  318  478  288  99  49  160  190  1685  
475   .813   29   93  122  231  82   4   80   84   383  
477   .711   36   50   86   24  25   3   15   47   219  

[14509 rows x 30 columns]

In [4]:
# Saving the DataFrame into a csv file.
stats_df.to_csv('NBA_Raw_data.csv', index=False)